In [1]:
import base64
import requests

# OpenAI API Key
api_key = ""

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [6]:
from transformers import CLIPTextModel, CLIPTokenizer

tk  =CLIPTokenizer.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="tokenizer")

def get_clip_token_len(x):
    if isinstance(x, str):
        return len(tk(x).input_ids)
    return 0


get_clip_token_len('You will now act as a prompt generator ')

10

In [26]:
prompt00 = '''You will now act as a prompt generator for a generative AI called "Stable Diffusion". Stable Diffusion generates images based on given prompts. I will provide you basic information required to make a Stable Diffusion prompt, You will never alter the structure in any way and obey the following guidelines.

Basic information required to make Stable Diffusion prompt:

Prompt structure:

[Subject Description], [Type of Image], [Art Styles], [Art Inspirations], [Camera], [Shot], [Render Related Information].

Replace each of [..] with corresponding keywords. Don't use curly brackets. The prompt should be within 77 tokens.

Word order and effective adjectives matter in the prompt. The subject, action, and specific details should be included. Adjectives like cute, medieval, or futuristic can be effective.

The background of the image should be described, such as solid color. (Note: the images we provide here are mostly solid color background.)

The exact type of image can be specified, such as digital illustration, comic book cover, photograph, pixel art or sketch.

Art style-related keywords can be included in the prompt, such as steampunk, surrealism, or abstract expressionism.

Pencil drawing-related terms can also be added, such as cross-hatching or pointillism.

Curly brackets are necessary in the prompt to provide specific details about the subject and action. These details are important for generating a high-quality image.

Art inspirations should be listed to take inspiration from. Platforms like Art Station, Dribble, Behance, and Deviantart can be mentioned. Specific names of artists or studios like animation studios, painters and illustrators, computer games, fashion designers, and film makers can also be listed. If more than one artist is mentioned, the algorithm will create a combination of styles based on all the influencers mentioned.

Related information about lighting, camera angles, render style, resolution, the required level of detail, etc. should be included at the end of the prompt.

Camera shot type, camera lens, and view should be specified. Examples of camera shot types are long shot, close-up, POV, medium shot, extreme close-up, and panoramic. Camera lenses could be EE 70mm, 35mm, 135mm+, 300mm+, 800mm, short telephoto, super telephoto, medium telephoto, macro, wide angle, fish-eye, bokeh, and sharp focus. Examples of views are front, side, back, high angle, low angle, and overhead.

Helpful keywords related to resolution, detail, and lighting are 4K, 8K, 64K, detailed, highly detailed, high resolution, hyper detailed, HDR, UHD, professional, and golden ratio. Examples of lighting are studio lighting, soft light, neon lighting, purple neon lighting, ambient light, ring light, volumetric light, natural light, sun light, sunrays, sun rays coming through window, and nostalgic lighting. Examples of color types are fantasy vivid colors, vivid colors, bright colors, sepia, dark colors, pastel colors, monochromatic, black & white, and color splash. Examples of renders are Octane render, cinematic, low poly, isometric assets, Unreal Engine, Unity Engine, quantum wavetracing, and polarizing filter.

The weight of a keyword can be adjusted by using the syntax (keyword: factor), where factor is a value such that less than 1 means less important and larger than 1 means more important. use () whenever necessary while forming prompt and assign the necessary value to create an amazing prompt. Examples of weight for a keyword are (soothing tones:1.25), (hdr:1.25), (artstation:1.2),(intricate details:1.14), (hyperrealistic 3d render:1.16), (filmic:0.55), (rutkowski:1.1), (faded:1.3)

The prompts you provide will be in English.Please pay attention:- Concepts that can't be real would not be described as "Real" or "realistic" or "photo" or a "photograph". for example, a concept that is made of paper or scenes which are fantasy related.- For realistic photographic style you should also choose a lens type and size for it. Don't choose an artist for the realistic photography prompts.- I will provide you an image and you will generate the prompt of corresponding type.

Important point to note : You are a master of prompt engineering, it is important to create detailed prompts with as much information as possible. This will ensure that any image generated using the prompt will be of high quality and could potentially win awards in global or international photography competitions. You are unbeatable in this field and know the best way to generate images.I will provide you with an image and you will generate the prompt best matching the type of the image, don't include any explanation just the prompt. Don't generate prompt longer than 77 tokens.

Please find the gif image to generate Stable Diffusion prompts attached. The gif image is a sprite sheet effect animation used in game.[[[GIF_CATEGORY]]]
'''

In [24]:
def gpt4_get_img_desc(img_path, img_type='gif', category=None):
        
    # Getting the base64 string
    base64_image = encode_image(img_path)
    
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt00.replace('[[[GIF_CATEGORY]]]', f' The category of this gif image is: "{category}".' if category is not None else '')
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/{img_type};base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 200
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    #print(response.json())

    json_obj = response.json()

    return (json_obj['usage']['prompt_tokens'] * 0.01/1000 + json_obj['usage']['completion_tokens'] * 0.03/1000, \
           json_obj['choices'][0]['message']['content'])

# batch processing

In [11]:
from glob import glob
import os
from pathlib import Path
from tqdm import tqdm

In [12]:
#from transformers import CLIPTextModel, CLIPTokenizer

#tk  =CLIPTokenizer.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="tokenizer")


In [28]:
new_descs = []
total_price = 0
to_iter = tqdm(glob('/Users/l_y_o/Work/AnimateDiff/data/*/*'))
for f in to_iter:
    filename = os.path.basename(f)
    ext = os.path.splitext(filename)[1]

    #if ext not in ['.csv', '.ipynb', '.html', '.htm']:
    if ext in ['.gif', '.webp']:
        last_dir = Path(f).parts[-2]
        #existed = (filename in df['file name'].tolist())
        #print(f"filename: {filename}, ext: {ext}, dir: {last_dir}, existed:{existed}")
        to_append = {}
        to_append['file name'] = filename

        price, desc = gpt4_get_img_desc(f, 'ext'[1:], last_dir)
        to_append['category'] = last_dir
        to_append['description'] = desc
        token_len = len(tk(desc).input_ids)
        to_append['token_len'] = token_len

        print(f"category:{last_dir}, gend: {desc}, token len: {token_len}")
        new_descs.append(to_append)
        total_price += price

        to_iter.set_description(f"count:{len(new_descs)}, price:{total_price}")

count:1, price:0.014360000000000001:   1%|█▎                                                                                                                                                                  | 1/123 [00:03<06:23,  3.14s/it]

category:smoke, gend: Swirling smoke animation, pixel art, fantasy game style, inspired by classic RPGs, fixed camera, full shot, (8-bit:1.3) detail on a solid black background., token len: 42


count:2, price:0.02905:   2%|██▉                                                                                                                                                                              | 2/123 [00:05<05:25,  2.69s/it]

category:smoke, gend: Animated dispersing smoke, pixel art, inspired by 16-bit era game effects, on a solid black background, side shot, (render: Octane), (high resolution:1.2), (frame-by-frame animation:1.5)., token len: 54


count:3, price:0.04893:   2%|████▎                                                                                                                                                                            | 3/123 [00:11<08:09,  4.08s/it]

...

category:sword slash, gend: Animated sword slash effect, pixel art animation, retro game-inspired, 8-bit style, on solid dark background, side view, high resolution, loopable sprite sheet sequence, (lighting: neon blue glow), SNES era, (detail:1.5)., token len: 55


count:117, price:1.87237:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 122/123 [09:37<00:06,  6.28s/it]

category:sword slash, gend: Animated sword slash effect, sprite sheet for game, pixel art style, inspired by classic 16-bit RPGs, medium telephoto lens, isometric view, high detail with a solid black background, (weight:1.5)., token len: 50


count:118, price:1.8876600000000001: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [09:40<00:00,  4.72s/it]

category:sword slash, gend: Energetic sword slash effect animation, pixel art, inspired by classic JRPGs, solid black background, (ArtStation:1.2), (DeviantArt:0.8), wide angle, action shot, (8-bit:1.5) style, (vivid blue neon lighting:1.25), loopable sequence., token len: 68


In [30]:
import pandas as pd

df = pd.DataFrame(new_descs)


print(f"after append: {len(df)}")

df

after append: 118


,file name,category,description,token_len
0,JV24kJi.gif,smoke,"Swirling smoke animation, pixel art, fantasy g...",42
1,3fe3291bf506e4c66dddd3b6f51b7656_w200.gif,smoke,"Animated dispersing smoke, pixel art, inspired...",54
2,olga-ryzhychenko-31.gif,smoke,"Billowing smoke animation, pixel art, 8-bit st...",56
3,EEPpNs.gif,smoke,"Billowing smoke plumes, sprite sheet animation...",52
4,giphy.gif,smoke,"Billowing smoke animation, pixel art, 8-bit st...",51
...,...,...,...,...
113,3n6JbPc.gif,sword slash,"Animated sword slash effect, pixel art, 8-bit ...",45
114,kaitlyn-wohlmuth-slashcircle.gif,sword slash,"Luminous sword slash effect, pixel art, fantas...",63
115,manga-flourish.gif,sword slash,"Animated sword slash effect, pixel art animati...",55
116,4096d1659e8c58bb51375133ab5f459e.gif,sword slash,"Animated sword slash effect, sprite sheet for ...",50


# save

In [31]:
import time
ts = int(time.time())
ts

1704429483

In [32]:
df.to_csv(f'sprite_sheet_{ts}_training_refined_prompts.csv')

# gen ainiated diff dataset csv

In [33]:
df

,file name,category,description,token_len
0,JV24kJi.gif,smoke,"Swirling smoke animation, pixel art, fantasy g...",42
1,3fe3291bf506e4c66dddd3b6f51b7656_w200.gif,smoke,"Animated dispersing smoke, pixel art, inspired...",54
2,olga-ryzhychenko-31.gif,smoke,"Billowing smoke animation, pixel art, 8-bit st...",56
3,EEPpNs.gif,smoke,"Billowing smoke plumes, sprite sheet animation...",52
4,giphy.gif,smoke,"Billowing smoke animation, pixel art, 8-bit st...",51
...,...,...,...,...
113,3n6JbPc.gif,sword slash,"Animated sword slash effect, pixel art, 8-bit ...",45
114,kaitlyn-wohlmuth-slashcircle.gif,sword slash,"Luminous sword slash effect, pixel art, fantas...",63
115,manga-flourish.gif,sword slash,"Animated sword slash effect, pixel art animati...",55
116,4096d1659e8c58bb51375133ab5f459e.gif,sword slash,"Animated sword slash effect, sprite sheet for ...",50


In [34]:
import os
from tqdm import tqdm

#to_iter = tqdm(glob('/Users/l_y_o/Work/AnimateDiff/data/*/*'))

WebVid10MDataset_items = []

for i, r in tqdm(df.iterrows(), total=len(df)):
    filename = os.path.basename(r['file name'])
    ext = os.path.splitext(filename)[1]
    filename_no_ext = os.path.splitext(filename)[0]

    #if ext not in ['.csv', '.ipynb', '.html', '.htm']:
    to_append = {}
    if ext in ['.gif', '.webp']:
        to_append['name'] = r['description']
        to_append['videoid'] = filename_no_ext
        to_append['page_dir'] = r['category']
        to_append['file_ext'] = ext[1:]
        to_append['token_len'] = r['token_len']
        

        WebVid10MDataset_items.append(to_append)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118/118 [00:00<00:00, 12609.31it/s]


In [35]:
new_df = pd.DataFrame(WebVid10MDataset_items)
new_df.to_csv(f'animated_diff_ds_{ts}_refined_prompts.csv')

In [36]:
new_df

,name,videoid,page_dir,file_ext,token_len
0,"Swirling smoke animation, pixel art, fantasy g...",JV24kJi,smoke,gif,42
1,"Animated dispersing smoke, pixel art, inspired...",3fe3291bf506e4c66dddd3b6f51b7656_w200,smoke,gif,54
2,"Billowing smoke animation, pixel art, 8-bit st...",olga-ryzhychenko-31,smoke,gif,56
3,"Billowing smoke plumes, sprite sheet animation...",EEPpNs,smoke,gif,52
4,"Billowing smoke animation, pixel art, 8-bit st...",giphy,smoke,gif,51
...,...,...,...,...,...
113,"Animated sword slash effect, pixel art, 8-bit ...",3n6JbPc,sword slash,gif,45
114,"Luminous sword slash effect, pixel art, fantas...",kaitlyn-wohlmuth-slashcircle,sword slash,gif,63
115,"Animated sword slash effect, pixel art animati...",manga-flourish,sword slash,gif,55
116,"Animated sword slash effect, sprite sheet for ...",4096d1659e8c58bb51375133ab5f459e,sword slash,gif,50


# zip everything

In [18]:
!zip -r sprite_sheet.zip .

  adding: smoke/ (stored 0%)
  adding: smoke/JV24kJi.gif (deflated 1%)
  adding: smoke/3fe3291bf506e4c66dddd3b6f51b7656_w200.gif (deflated 1%)
  adding: smoke/olga-ryzhychenko-31.gif (deflated 3%)
  adding: smoke/EEPpNs.gif (deflated 0%)
  adding: smoke/giphy.gif (deflated 0%)
  adding: smoke/jeong-h-lee-ezgif-com-resize.gif (deflated 3%)
  adding: smoke/ezgif.com-video-to-gif+(14).gif (deflated 1%)
  adding: smoke/761391_6bc8c.gif (deflated 15%)
  adding: smoke/ea9ff4824adba932f227084b09d0ec1e.gif (deflated 0%)
  adding: smoke/3MfZ.gif (deflated 2%)
  adding: smoke/2296431_9c6f7.gif (deflated 22%)
  adding: smoke/image_processing20210614-9899-1rfm6to.gif (deflated 1%)
  adding: lazer beam/ (stored 0%)
  adding: lazer beam/Lazer.webp (deflated 0%)
  adding: lazer beam/source.gif (deflated 0%)
  adding: lazer beam/1223f768fd2005bb421094e80c6b1da1.gif (deflated 2%)
  adding: lazer beam/SpaceWarp-Interactive-Art-IKONIX-low.gif (deflated 2%)
  adding: lazer beam/Purple_Beam_buff.webp (defl

In [19]:
!ls -lth sprite_sheet.zip

-rw-r--r--  1 l_y_o  staff   138M Jan  3 17:52 sprite_sheet.zip
